In [1]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
new_data=pd.read_csv("../input/csvvvv/Classeur 1 (5).csv")

In [3]:
new_data.drop(["Y","Word Count","Language",' '], axis=1, inplace=True)
new_data.columns = ["text", "CEFR Review","CEFR Level_Samar"]

In [4]:
new_data.shape

(191, 3)

In [5]:
new_data.head(2)

,text,CEFR Review,CEFR Level_Samar
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2,C1
1,Quoique ce détail ne touche en aucune manière ...,C1,C2


In [6]:
mike_20_june_samar=new_data[pd.notnull(new_data['CEFR Level_Samar'])]
mike_20_june_samar.columns = ["text", "CEFR Review","label"]
mike_20_june_samar.drop(["CEFR Review"],axis=1, inplace=True)
mike_20_june_samar.to_csv("mike_20_june_samar.csv" , index=False)
mike_20_june_samar.head()

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",C1
1,Quoique ce détail ne touche en aucune manière ...,C2
2,"En 1804, M. Myriel était curé de Brignolles. I...",C1
3,"Vers l'époque du couronnement, une petite affa...",C2
4,—Quel est ce bonhomme qui me regarde?,C1


In [7]:
mike_20_june_new=new_data[pd.notnull(new_data['CEFR Review'])]
mike_20_june_new.columns = ["text", "label","CEFR Level_Samar"]
mike_20_june_new.drop(["CEFR Level_Samar"],axis=1, inplace=True)
mike_20_june_new.to_csv("mike_20_june_new.csv" , index=False)
mike_20_june_new.head() 

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2
1,Quoique ce détail ne touche en aucune manière ...,C1
2,"En 1804, M. Myriel était curé de Brignolles. I...",A2
3,"Vers l'époque du couronnement, une petite affa...",B2
4,—Quel est ce bonhomme qui me regarde?,A2


**!!!! samar annotation seems to be weird because there is just C1 and C2**

In [8]:
mike_20_june_samar["label"].unique(), mike_20_june_new["label"].unique()

(array(['C1', 'C2'], dtype=object),
 array(['A2', 'C1', 'B2', 'A1', 'B1', 'C2'], dtype=object))

In [9]:
mike_20_june_new.label.value_counts()

C1    58
A1    41
B2    33
B1    29
A2    24
C2     6
Name: label, dtype: int64

In [10]:
mike_20_june_samar.shape,mike_20_june_new.shape

((141, 2), (191, 2))

In [11]:
trainn=pd.read_csv("../input/frenchcefr/french dataset.csv")
trainn.shape

(315, 4)

In [12]:
text=[]
label=[]
for i in range(trainn.shape[0]):
    if (trainn.loc[i,"label"]=="A1") | (trainn.loc[i,"label"]=="A2"):
        text.append(trainn.loc[i,"text"])
        label.append(trainn.loc[i,"label"])

        

In [13]:
trainn.head(1)

,text,len,lang,label
0,Il était une fois un prince qui voulait épouse...,15,FR,A2


In [14]:
len(text), len(label)

(96, 96)

In [15]:
text[0], label[0]

('Il était une fois un prince qui voulait épouser une princesse, mais une vraie princesse.',
 'A2')

****

In [16]:
A1_A2=pd.DataFrame({"text":text,"label":label})

In [17]:
A1_A2.to_csv("A1_A2.csv", index=False)

****

In [18]:
data=pd.read_csv("../input/eng-data/translated.csv")

In [19]:
data=pd.read_csv("../input/eng-data/translated.csv")
textt=[]
labell=[]
for i in range(data.shape[0]):
    if (data.loc[i,"label"]=="A1") | (data.loc[i,"label"]=="A2"):
        textt.append(data.loc[i,"text"])
        labell.append(data.loc[i,"label"])
A1_A2_tran=pd.DataFrame({"text":textt,"label":labell})


In [20]:
txt=[]
label=[]
nb=0
nbb=0
for i in range(A1_A2_tran.shape[0]):
    if A1_A2_tran.loc[i,"label"]=="A2":
        if nb> 30 :
            continue
        txt.append(A1_A2_tran.loc[i,"text"])
        label.append(A1_A2_tran.loc[i,"label"])
        nb+=1
       
    elif  A1_A2_tran.loc[i,"label"]=="A1": 
            if nbb> 30 :
                continue
    
            txt.append(A1_A2_tran.loc[i,"text"])
            label.append(A1_A2_tran.loc[i,"label"])
            nbb+=1
        
    if nbb> 30 and nb> 30:
        break

In [21]:
A1_A2_30=pd.DataFrame({"text":txt,"label":label})

In [22]:
A1_A2_30.to_csv("A1_A2_30.csv", index=False)

In [23]:
A1_A2_tran.to_csv("A1_A2_tran.csv", index=False)

****

In [24]:
dataa=pd.read_csv("../input/lingua/fr_lingua.csv")
textt=[]
labell=[]
for i in range(dataa.shape[0]):
    if (dataa.loc[i,"cefr"]=="A1") | (dataa.loc[i,"cefr"]=="A2"):
        textt.append(dataa.loc[i,"text"])
        labell.append(dataa.loc[i,"cefr"])
A1_A2_tran=pd.DataFrame({"text":textt,"label":labell})

In [25]:
A1_A2_tran.to_csv("A1_A2_lingua.csv", index=False)

****

In [26]:
dataa=pd.read_csv("../input/newdatasets/mike_20_june_new (1).csv")
textt=[]
labell=[]
for i in range(dataa.shape[0]):
    if (dataa.loc[i,"label"]=="A1") | (dataa.loc[i,"label"]=="A2"):
        textt.append(dataa.loc[i,"text"])
        labell.append(dataa.loc[i,"label"])
A1_A2_tran=pd.DataFrame({"text":textt,"label":labell})

In [27]:
A1_A2_tran.to_csv("A1_A2_mike_new.csv", index=False)